<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=78fb9cd870d2683b4c0e20972d0c94d864a2a67e6b49d3ab8877541ad7a06420
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-09 15:02:37
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.40 C
-------------------
Today PnL: 61.05 K (0.44%)
Current PnL: -18.86 L (-12.68%)
CY Booked + Current PnL: -7.83 L (-5.27%)
-------------------
Total profit:  1.72 L
Total loss:  -20.58 L
-------------------
Total Booked + Current PnL: 19.15 L (15.74%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.86%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.51 L (55.93%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.95%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.75,59.0,H-LC,10.79,172370.0,16193.0,9894.0,0.62,10.37,5.74,16.71,22.0,1.64,1.33,29.61,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,106.19,64.0,M-SC,5.88,88805.0,-11972.0,12069.0,0.84,-11.88,13.59,0.09,245.0,-0.99,0.69,16.67,OX40N,NTT,DURABLES
62,SBIN,863.00,-13.25,48.0,M-LC,4.29,214398.0,12918.0,14300.0,0.03,6.41,6.67,13.51,60.0,0.90,1.66,19.82,XY25,NTT,BANKS
41,ITC,452.00,-37.92,51.0,H-LC,3.22,200588.0,450.0,20440.0,0.65,0.22,10.19,10.44,4.0,0.02,1.55,5.80,X40,NTT,FMCG
49,NATIONALUM,247.44,-45.32,66.0,H-MC,1.72,109805.0,9301.0,21335.0,-1.61,9.25,19.43,30.49,79.0,0.44,0.85,44.66,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ANGELONE,3033.00,8.67,40.0,H-SC,6.53,191962.0,956.0,56744.0,1.92,0.50,29.56,30.21,157.0,0.02,1.48,21.49,X40N,NTT,FINANCE
35,ICICIPRULI,790.00,-21.45,39.0,H-MC,2.07,135216.0,-572.0,43323.0,0.87,-0.42,32.04,31.48,107.0,-0.01,1.05,11.65,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.23,61.0,H-MC,1.80,247166.0,-565.0,75633.0,0.33,-0.23,30.60,30.30,92.0,-0.01,1.91,13.03,X40,ATH,ELECTRICAL
34,ICICIGI,2260.25,-21.04,40.0,H-MC,3.72,135783.0,-350.0,31474.0,0.10,-0.26,23.18,22.86,91.0,-0.01,1.05,12.32,X40,ATH,INSURANCE
51,PGHH,17973.08,-30.41,54.0,H-MC,4.79,201360.0,540.0,68241.0,1.11,0.27,33.89,34.25,80.0,0.01,1.56,5.48,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,JIOFIN,387.00,-2.61,38.0,H-LC,13.87,215320.0,-2086.0,55574.0,-1.13,-0.96,25.81,24.61,48.0,-0.04,1.66,53.33,XY24,BTT,FINANCE
31,HCLTECH,1943.91,-1.94,38.0,H-LC,10.16,224165.0,-17759.0,81036.0,1.77,-7.34,36.15,26.15,8.0,-0.22,1.73,5.93,X40,ATH,IT
83,VBL,671.64,-16.13,38.0,H-LC,6.66,302252.0,-13590.0,126251.0,-0.06,-4.30,41.77,35.67,5.0,-0.11,2.34,8.96,X40N,ATH,FMCG
74,TCS,4998.00,-28.04,43.0,H-LC,14.19,286681.0,-59275.0,183132.0,1.02,-17.13,63.88,35.80,1.0,-0.32,2.22,1.56,X40,BTT,IT
80,UNITDSPR,1644.00,-14.51,44.0,H-LC,8.34,171890.0,-9869.0,45121.0,0.48,-5.43,26.25,19.39,86.0,-0.22,1.33,2.28,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-5.78,52.0,H-LC,1.60,233348.0,-12803.0,92009.0,1.90,-5.20,39.43,32.18,16.0,-0.14,1.80,28.97,X200,ATH,IT
50,NESTLEIND,1377.00,-8.60,59.0,H-LC,2.53,281385.0,15959.0,40829.0,1.25,6.01,14.51,21.40,11.0,0.39,2.18,13.78,XY25,NTT,FMCG
41,ITC,452.00,-37.92,51.0,H-LC,3.22,200588.0,450.0,20440.0,0.65,0.22,10.19,10.44,4.0,0.02,1.55,5.80,X40,NTT,FMCG
20,CIPLA,1795.00,-20.47,49.0,H-LC,5.80,212446.0,7946.0,33460.0,0.62,3.89,15.75,20.25,10.0,0.24,1.64,12.96,X40N,BTT,PHARMA
83,VBL,671.64,-16.13,38.0,H-LC,6.66,302252.0,-13590.0,126251.0,-0.06,-4.30,41.77,35.67,5.0,-0.11,2.34,8.96,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,103.03,69.0,H-SC,12.87,128142.0,-14337.0,33855.0,7.45,-10.06,26.42,13.70,163.0,-0.42,0.99,51.24,OX40N,NTT,BANKS
46,LAOPALA,464.0,120.35,56.0,H-SC,3.21,69967.0,-30879.0,56701.0,5.36,-30.62,81.04,25.61,142.0,-0.54,0.54,29.72,AR,NTT,CERAMICS
39,INFY,2275.0,-17.80,53.0,H-LC,10.55,320160.0,7272.0,164402.0,4.90,2.32,51.35,54.87,3.0,0.04,2.48,9.13,X40,BTT,IT
81,VAIBHAVGBL,521.0,54.82,64.0,H-SC,11.09,103231.0,-48760.0,122886.0,4.18,-32.08,119.04,48.77,125.0,-0.40,0.80,20.16,XR,NTT,JEWELLERY
75,TITAGARH,1548.0,-6.04,58.0,H-SC,7.83,153912.0,-38103.0,116989.0,4.08,-19.84,76.01,41.08,158.0,-0.33,1.19,30.42,XY24,NTT,ENGINEERING


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3727.03,64.0,M-SC,7.79,119357.0,-28011.0,79826.0,-5.50,-19.01,66.88,35.16,236.0,-0.35,0.92,21.91,XY24,NTT,AUTO
22,COFFEEDAY,80.0,-39.64,59.0,L-SC,40.38,89856.0,-23693.0,63744.0,-4.12,-20.87,70.94,35.27,268.0,-0.37,0.69,119.92,XR,NTT,HOTELS
68,STARHEALTH,761.0,7.79,50.0,H-SC,3.72,160137.0,-40079.0,116099.0,-2.31,-20.02,72.50,37.97,144.0,-0.35,1.24,28.58,XY24,NTT,INSURANCE
67,SONACOMS,1006.0,-37.62,46.0,M-SC,12.34,77718.0,-23441.0,98337.0,-2.26,-23.17,126.53,74.03,202.0,-0.24,0.60,9.59,AR,BTT,AUTO
14,BERGEPAINT,680.0,-20.05,41.0,H-MC,7.36,214711.0,-12628.0,60699.0,-1.77,-5.55,28.27,21.14,106.0,-0.21,1.66,21.02,XY24,NTT,PAINTS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,103.03,69.0,H-SC,12.87,128142.0,-14337.0,33855.0,7.45,-10.06,26.42,13.70,163.0,-0.42,0.99,51.24,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,106.19,64.0,M-SC,5.88,88805.0,-11972.0,12069.0,0.84,-11.88,13.59,0.09,245.0,-0.99,0.69,16.67,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.42,63.0,H-SC,2.60,224082.0,-45585.0,81924.0,2.33,-16.90,36.56,13.47,138.0,-0.56,1.73,13.34,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,113.17,47.0,M-SC,3.26,139686.0,-34873.0,34908.0,-1.32,-19.98,24.99,0.02,221.0,-1.00,1.08,21.50,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.05,45.0,H-SC,2.41,114111.0,-29692.0,56439.0,-0.70,-20.65,49.46,18.60,149.0,-0.53,0.88,25.92,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-28.04,43.0,H-LC,14.19,286681.0,-59275.0,183132.0,1.02,-17.13,63.88,35.80,1.0,-0.32,2.22,1.56,X40,BTT,IT
39,INFY,2275.00,-17.80,53.0,H-LC,10.55,320160.0,7272.0,164402.0,4.90,2.32,51.35,54.87,3.0,0.04,2.48,9.13,X40,BTT,IT
41,ITC,452.00,-37.92,51.0,H-LC,3.22,200588.0,450.0,20440.0,0.65,0.22,10.19,10.44,4.0,0.02,1.55,5.80,X40,NTT,FMCG
83,VBL,671.64,-16.13,38.0,H-LC,6.66,302252.0,-13590.0,126251.0,-0.06,-4.30,41.77,35.67,5.0,-0.11,2.34,8.96,X40N,ATH,FMCG
1,ABB,7934.00,-41.13,45.0,H-LC,12.75,246187.0,-15432.0,134640.0,0.40,-5.90,54.69,45.57,7.0,-0.11,1.90,4.75,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,9.32,53.0,H-MC,3.36,188730.0,24060.0,43899.0,0.16,14.61,23.26,41.28,89.0,0.55,1.46,23.76,X40N,ATH,AC
4,ANGELONE,3033.00,8.67,40.0,H-SC,6.53,191962.0,956.0,56744.0,1.92,0.50,29.56,30.21,157.0,0.02,1.48,21.49,X40N,NTT,FINANCE
84,VOLTAS,1918.49,1.02,59.0,H-MC,3.48,211800.0,20058.0,75973.0,-0.51,10.46,35.87,50.08,99.0,0.26,1.64,17.97,XY25,ATH,AC
24,DABUR,735.00,-2.58,61.0,H-MC,5.14,211887.0,14859.0,74754.0,-0.53,7.54,35.28,45.49,102.0,0.20,1.64,21.57,XY24,BTT,FMCG
18,CAMS,5211.76,-5.16,51.0,H-SC,5.14,107803.0,5799.0,38130.0,0.13,5.69,35.37,43.06,122.0,0.15,0.83,23.77,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-45.32,66.0,H-MC,1.72,109805.0,9301.0,21335.0,-1.61,9.25,19.43,30.49,79.0,0.44,0.85,44.66,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.15,63.0,M-SC,3.11,102810.0,11312.0,67443.0,0.57,12.36,65.60,86.07,223.0,0.17,0.79,49.30,XR,NTT,DURABLES
36,INDIAMART,4850.92,-51.09,49.0,H-SC,12.12,136380.0,13044.0,120724.0,-0.05,10.58,88.52,108.45,119.0,0.11,1.05,35.29,AR,ATH,MISC
86,WIPRO,420.00,-11.98,51.0,M-LC,6.75,154405.0,3460.0,105999.0,2.69,2.29,68.65,72.51,53.0,0.03,1.19,8.45,XR,NTT,IT
12,BANKINDIA,190.00,-32.92,48.0,H-MC,6.58,173333.0,-6475.0,118889.0,-0.44,-3.60,68.59,62.52,88.0,-0.05,1.34,29.26,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6716.67,52.0,L-SC,6.20,74814.0,-18996.0,95612.0,1.28,-20.25,127.80,81.67,269.0,-0.20,0.58,44.88,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-6.19,40.0,H-SC,14.99,87290.0,-13673.0,108205.0,-0.27,-13.54,123.96,93.63,148.0,-0.13,0.67,30.02,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,59.0,L-SC,40.38,89856.0,-23693.0,63744.0,-4.12,-20.87,70.94,35.27,268.0,-0.37,0.69,119.92,XR,NTT,HOTELS
48,MASFIN,398.61,-15.99,60.0,H-SC,12.95,95415.0,-2565.0,24169.0,2.48,-2.62,25.33,22.05,152.0,-0.11,0.74,38.69,XR,ATH,FINANCE
82,VALIANTORG,838.00,-335.81,64.0,H-SC,8.88,100126.0,-33160.0,117758.0,3.35,-24.88,117.61,63.47,139.0,-0.28,0.77,67.84,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-39.64,59.0,L-SC,40.38,89856.0,-23693.0,63744.0,-4.12,-20.87,70.94,35.27,268.0,-0.37,0.69,119.92,XR,NTT,HOTELS
8,ATULAUTO,844.0,3727.03,64.0,M-SC,7.79,119357.0,-28011.0,79826.0,-5.50,-19.01,66.88,35.16,236.0,-0.35,0.92,21.91,XY24,NTT,AUTO
60,SAMMAANCAP,326.0,-171.59,63.0,M-SC,4.60,81864.0,-20346.0,113734.0,-0.93,-19.91,138.93,91.37,208.0,-0.18,0.63,32.89,XY25,NTT,FINANCE
82,VALIANTORG,838.0,-335.81,64.0,H-SC,8.88,100126.0,-33160.0,117758.0,3.35,-24.88,117.61,63.47,139.0,-0.28,0.77,67.84,XR,NTT,CHEMICALS
55,RELAXO,1176.0,-31.65,60.0,H-SC,3.38,63398.0,-52140.0,87128.0,1.38,-45.13,137.43,30.28,136.0,-0.60,0.49,24.64,X40N,NTT,FOOTWEAR


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.31
1,25,44.49
2,50,75.05


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.87
LC    35.33
MC    23.79
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.69
X40      16.06
X40N     13.15
XY25     10.33
XR        9.51
AR        8.32
OX40N     7.78
X200      1.80
MH        1.78
X5K       1.46
SR        1.11
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.77
H-SC    24.98
H-MC    20.70
M-SC    14.21
M-LC     7.45
M-MC     2.76
L-SC     1.68
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,12.79,-17.86,77.53
FMCG,12.36,-3.92,39.15
FINANCE,9.07,-19.02,61.82
BANKS,7.68,-14.88,63.66
PAINTS,6.19,-15.22,41.01
MISC,6.04,-17.64,85.08
ELECTRICAL,5.87,-8.44,68.20
AC,3.85,2.69,30.27
AUTO,3.54,-14.58,66.25


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2753927.0,22
AR,1193817.0,9
XR,1062937.0,13
X40,890811.0,10
X40N,574631.0,10
OX40N,452773.0,10
XY25,416677.0,7
SR,199173.0,2
X5K,129101.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2791456.0,28
M-SC,1407308.0,17
H-LC,1401773.0,15
H-MC,1169495.0,15
M-LC,415587.0,5
M-MC,320180.0,2
L-SC,244158.0,3
L-MC,60084.0,1
L-LC,40676.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,926307.0,6
M-SC,XY24,818335.0,7
H-SC,AR,658486.0,5
H-LC,X40,600100.0,5
H-SC,XR,502085.0,6
H-MC,XY24,467344.0,4
H-LC,AR,374100.0,2
M-MC,XY24,320180.0,2
H-SC,X40N,250141.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
